# Include all moudels used in all parts......

In [14]:
import numpy as np
import cv2
import mediapipe as mp
import math
from sklearn import svm,preprocessing

# A class that extracts the Features of a hand......

In [15]:
class HandFetuers:

    #Method Get Vector From 2 Points
    @classmethod
    def ConvertPoints2Vector(cls,p1,p2):
        vector=p2-p1
        vector[1]=-vector[1]
        return vector
    #...........................

    #Method Get Lenghte Vector
    @classmethod
    def GetLengthVector(cls,vector):
        return math.sqrt(np.dot(vector,vector))
    #...........................

    #Method Get Coner Between 2 Vectors
    @classmethod
    def GetConerBetween2Vector(cls,v1,v2):
        return  math.pi-math.acos((np.dot(v1,v2)/(cls.GetLengthVector(v1)*cls.GetLengthVector(v2))))
    #...........................

    #Method  Get Vectors Fingers Hand 16x2  4 for finger Thume and  3 echo fingers
    @classmethod 
    def GetVectorsFingers(cls,Points):
        FingersVectors=np.zeros((16,2))
        i,j=0,0
        while i<21:
            FingersVectors[j]=cls.ConvertPoints2Vector(Points[i],Points[i+1])
            j=j+1
            i=i+1+int((i+1)%4==0)
        return FingersVectors     
    #...........................

    # Method Get Conner For Aall Fingers Vectors Retuern 16x1.  4 coners for Finger Thumes and 3 Coners echo fingers 
    @classmethod
    def GetConnersFingers(cls,FingersVectors,Points):
        ConersFingersVectors=np.zeros((16,1))
        i,f=0,1
        while i<3:
            ConersFingersVectors[i]=cls.GetConerBetween2Vector(FingersVectors[i],FingersVectors[i+1])
            i=i+1
        ConersFingersVectors[i]=cls.GetConerBetween2Vector(
                                                          cls.ConvertPoints2Vector(Points[1],Points[2]),
                                                          cls.ConvertPoints2Vector(Points[2],Points[4])
                                                          )
        i=i+1
        while i<16:
            ConersFingersVectors[i]=cls.GetConerBetween2Vector(FingersVectors[i],FingersVectors[i+1])
            if (i+1)%3==0:
                i=i+1
                ConersFingersVectors[i]=cls.GetConerBetween2Vector(
                                                            cls.ConvertPoints2Vector(Points[i-f],Points[i-f+1]),
                                                            cls.ConvertPoints2Vector(Points[i-f+1],Points[i-f+3])
                                                                  )
                f=f-1
            i=i+1   
        return  ConersFingersVectors   
    #...........................


    # Method Get Hand is Flipe Or NoFlipe return 1 for Flipe Or 0
    @classmethod 
    def CheckFlipeHand(cls,points,isHandLeft):
        A,B=points[17,:2]-points[0,:2],points[5,:2]-points[0,:2]
        B[1],A[1]=-B[1],-A[1]
        Bo=((math.pi*2)+math.atan2(B[1], B[0]))%(math.pi*2)
        Ao=((math.pi*2)+math.atan2(A[1], A[0]))%(math.pi*2)
        if (Ao<=math.pi/2) and (Bo>=math.radians(270)):
            Bo=-Bo
        elif (Bo<=math.pi/2) and (Ao>=math.radians(270)):
            Ao=-Ao
        lable=0
        if (Bo<Ao)^isHandLeft:
            lable=1        
        return lable 
    #...........................

    #Method Get Dirction Hand  Conner return Value Between 0 to 2*Pi
    @classmethod
    def GetHandDirction(cls,points):
        A=points[9,:2]-points[0,:2]
        A[1]=-A[1]
        theat=((2*math.pi)+math.atan2(A[1],A[0]))%(2*math.pi)
        return theat
    #...........................

    #Method Get Conner Between Thumes and All Landmark Fingers return 4x9 
    @classmethod
    def GetConnersBetweenThumesAndFingers(cls,points):
        p0,p5,p17=points[0,:2],points[5,:2],points[17,:2]
        alf1,alf2=1,2
        mp0p5=np.array([[(alf1*p5[0]+p0[0])/(1+alf1),(alf1*p5[1]+p0[1])/(1+alf1)]]).reshape(p0.shape)
        cm=np.array([[(alf2*mp0p5[0]+p17[0])/(1+alf2),(alf2*mp0p5[1]+p17[1])/(1+alf2)]]).reshape(p0.shape)
        res=np.zeros((4,9))
        j=6
        for i in range(4):
            f=0   
            for n in range(2,5,1):   
                for z in range(3):
                    v=cls.ConvertPoints2Vector(cm, points[j+z,:2])
                    vcmp4=cls.ConvertPoints2Vector(points[n,:2],cm)
                    res[i,f]=cls.GetConerBetween2Vector(v,vcmp4)
                    f=f+1
            j=j+1
        return res
    #...........................

    #Method Get All Feteuers return 1x54
    @classmethod
    def GetFeteuers(cls,landmrks,ishandleft):    
        points=np.zeros((21,2))
        i=0
        for pos in  landmrks:  
            points[i]=[pos.x,pos.y]
            i=i+1
        FingersVectors=cls.GetVectorsFingers(points)
        ConersFingersVectors=cls.GetConnersFingers(FingersVectors,Points=points)
        IsFlipeHand=cls.CheckFlipeHand(points,isHandLeft=ishandleft)
        DirctionHand=cls.GetHandDirction(points)
        ConersThumeAndAllFingers=cls.GetConnersBetweenThumesAndFingers(points)
         
        Fetuers=np.concatenate([np.array([[int(IsFlipeHand),DirctionHand]]),ConersFingersVectors.T],axis=1)
        Fetuers=np.concatenate([Fetuers,ConersThumeAndAllFingers.reshape((1,36))],axis=1)
        return Fetuers
    #...........................
#......................................................End Class..............................  
        

# A function that extracts all the features of a dataset and stores it in a file


In [8]:
import os  
def Fetuers_Extraction_And_Save(Path_Dataset,Path_File_Fetuers):
    hands=mp.solutions.hands.Hands(static_image_mode=True,max_num_hands=1,min_detection_confidence=0.7)
    index=0
    errorfile=0
    fileobj=open(Path_File_Fetuers,'a')
    for folder in [fd for fd in os.listdir(Path_Dataset) if os.path.isdir(Path_Dataset+'\\'+fd)]:
        path=Path_Dataset+"\\"+folder
        for file in [(path+'\\'+fi) for fi in os.listdir(path) if os.path.isfile(path+'\\'+fi)]:
            
            try:
                
                image=cv2.imread(file)
                image= cv2.flip(image, 1)
                res=hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))    
                if res.multi_hand_landmarks:
                    IsLeftHand=res.multi_handedness[0].classification[0].label!="Right"
                    Fetuers=HandFetuers.GetFeteuers(res.multi_hand_landmarks[0].landmark, IsLeftHand)
                    FetuersAll=np.append(Fetuers,[[ord(folder)]],axis=1)    
                    np.savetxt(fileobj,FetuersAll,fmt='%f',delimiter=';')
                    index=index+1
                    print("Proccessing Images = ",index)
                    
                   
                    
            except BaseException:
                errorfile=errorfile+1
                print('errror......')
                continue
                 
      
    fileobj.close()
    print('.............................\n',"Correct files numbers= ",index)
    print("Error files numbers= ",errorfile)
    del hands
    
#.......................................................End.....    
                

In [ ]:
Fetuers_Extraction_And_Save(Path_Dataset="SigNN Character Database",Path_File_Fetuers="Fetuers.txt")

# preprocessing

In [3]:
import pandas as pd

df=pd.read_csv("Fetuers.txt",sep=';',header=None)
df[df.columns[-1]]=df[df.columns[-1]].map(lambda x:chr(int(x))).astype(str)
print("Dimensions of the data set = ",df.shape)
X=df.iloc[:,:-1]
Y=np.asarray(df.iloc[:,-1])

df.sample(10)

Dimensions of the data set =  (8045, 55)


,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
567,0.0,1.698907,2.552134,2.319583,2.470574,2.001125,3.073766,3.074526,3.043076,3.113516,...,1.560829,1.752114,1.802123,0.802226,0.993512,1.043520,0.020259,0.171027,0.221035,B
5693,0.0,1.820215,2.153777,2.194279,2.938703,2.093632,3.118788,3.122021,3.108732,3.134161,...,2.092006,2.063171,2.059169,1.605909,1.577074,1.573072,0.635274,0.664109,0.668111,R
4,0.0,1.960183,2.335558,2.746706,3.008591,2.803339,0.022838,3.120188,0.013008,0.143165,...,1.303715,1.388305,1.076303,0.599417,0.684007,0.372005,0.356515,0.441106,0.129104,A
3713,0.0,1.539463,2.493526,2.712484,3.134855,2.715333,3.137312,3.069112,3.102919,3.093069,...,1.066939,1.254225,1.313098,0.268051,0.455338,0.514210,0.034515,0.221801,0.280674,K
1819,0.0,1.778383,2.249674,2.697838,1.861796,2.273932,0.503409,2.644436,0.165012,0.360374,...,1.743711,1.866811,1.725964,0.926393,1.049492,0.908646,0.531053,0.654153,0.513306,E
1149,0.0,1.980628,2.691368,2.776018,3.009629,2.710717,2.522476,2.277442,2.164996,2.209195,...,1.889594,1.969959,1.792837,1.605661,1.686026,1.508904,1.470567,1.550933,1.373810,C
7431,0.0,1.577904,2.429251,2.314498,2.760307,2.145228,3.118368,3.117596,3.106959,3.114837,...,1.779841,1.811683,1.826490,0.308796,0.340639,0.355445,0.612812,0.580970,0.566163,W
2426,0.0,1.671907,2.510757,2.690316,2.307099,2.282102,0.026265,3.068353,0.063175,3.113577,...,1.959455,2.026517,2.048017,1.365277,1.432339,1.453839,0.744515,0.811577,0.833077,F
3982,0.0,1.787866,2.769345,3.102467,2.884862,3.062144,2.990499,3.043302,2.945198,0.583905,...,1.661861,2.063765,2.092816,1.253968,1.655872,1.684923,1.127807,1.529712,1.558762,L
7761,0.0,1.521562,2.680194,3.120374,2.839748,2.995875,0.024689,3.124592,0.029048,0.007695,...,1.287457,1.337317,0.499857,0.930826,0.980686,0.143227,0.856470,0.906330,0.068870,Y


In [4]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
count,8045.000000,8045.000000,8045.000000,8045.000000,8045.000000,8045.000000,8045.000000,8045.000000,8045.000000,8045.000000,...,8045.000000,8045.000000,8045.000000,8045.000000,8045.000000,8045.000000,8045.000000,8045.000000,8045.000000,8045.000000
mean,0.032567,1.660366,2.542101,2.623766,2.698183,2.462334,1.925077,2.928244,1.839549,1.592935,...,0.570738,1.549511,1.559803,1.355568,0.851764,0.850616,0.671492,0.620417,0.570738,0.463500
std,0.177511,0.700842,0.194221,0.315834,0.386261,0.397183,1.312724,0.354255,1.340783,1.286291,...,0.409924,0.358481,0.352220,0.495167,0.517217,0.466012,0.493223,0.483664,0.409924,0.381971
min,0.000000,0.004956,0.726164,0.003318,0.337703,0.041859,0.000038,0.011335,0.000085,0.000058,...,0.000224,0.005090,0.018065,0.012919,0.000085,0.000224,0.000430,0.000014,0.000224,0.000026
25%,0.000000,1.540597,2.415103,2.416653,2.474770,2.197463,0.297002,2.915121,0.197930,0.270047,...,0.245383,1.284559,1.324710,0.995786,0.455090,0.520696,0.285458,0.247481,0.245383,0.184027
50%,0.000000,1.675588,2.534437,2.647440,2.834724,2.465901,2.886118,3.066849,2.799837,1.597971,...,0.474813,1.563380,1.549168,1.350437,0.777766,0.814283,0.584772,0.493898,0.474813,0.365980
75%,0.000000,1.775930,2.672595,2.847932,2.989226,2.789174,3.085786,3.111293,3.071858,3.075143,...,0.823354,1.771456,1.810425,1.733760,1.201466,1.127529,0.956013,0.944252,0.823354,0.672532
max,1.000000,6.270501,3.135495,3.141498,3.141585,3.141449,3.141589,3.141584,3.141501,3.141580,...,3.112894,3.111946,3.135986,3.132796,3.133575,3.135240,3.138441,3.110043,3.112894,3.125229


# Split Data set (test,train)

In [4]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=42)
print('size train=',len(y_train),'; size test=',len(y_test))

size train= 5631 ; size test= 2414


# Svm(Svc) Model training 

In [5]:
model_svm=svm.SVC(kernel='poly',probability=True)
model_svm.fit(x_train,y_train)

SVC(kernel='poly', probability=True)

# View a report on the Svm(Svc) Model

In [6]:
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline
#.......................................
ylable=[chr(i) for i in range(65,91)]
ylable.remove('Z')
ylable.remove('J')
y_predict=model_svm.predict(x_test)
print(metrics.classification_report(y_test,y_predict,target_names=ylable))
#.......................................






              precision    recall  f1-score   support

           A       0.95      0.99      0.97       170
           B       0.97      0.99      0.98       153
           C       0.96      1.00      0.98       114
           D       0.98      0.98      0.98       124
           E       0.99      0.92      0.95       138
           F       0.99      1.00      1.00       127
           G       1.00      0.99      1.00       101
           H       0.99      0.98      0.99       109
           I       0.99      1.00      0.99        97
           K       0.99      0.96      0.97        76
           L       0.99      1.00      0.99        98
           M       0.89      0.88      0.89        84
           N       0.87      0.81      0.84        89
           O       0.97      0.98      0.98       109
           P       1.00      1.00      1.00        54
           Q       1.00      0.97      0.98        63
           R       0.91      0.91      0.91        87
           S       0.86    

#  save the model in file

In [7]:
import pickle
pickle.dump(model_svm, open("SVM(SVC)_Model.sav", 'wb'))

# Load the model from file......

In [16]:
import pickle
model = pickle.load(open("SVM(SVC)_Model.sav", 'rb'))

# detction hand and write lable in image.....

In [17]:
def Detection_Image(Image_or_Frame):
    res=hands.process(cv2.cvtColor(Image_or_Frame, cv2.COLOR_BGR2RGB))    
    if res.multi_hand_landmarks:
            IsLeftHand=res.multi_handedness[0].classification[0].label=="Right"
            Fetuers=HandFetuers.GetFeteuers(res.multi_hand_landmarks[0].landmark, IsLeftHand) 
            mp_draw.draw_landmarks(Image_or_Frame, res.multi_hand_landmarks[0], mp.solutions.hands.HAND_CONNECTIONS,
                                   connection_drawing_spec=mp_draw.DrawingSpec(color=(0,255,0))
                                  )
            proba=model.predict_proba(Fetuers)[0]                        
            if max(proba)>=0.5:                     
                lable=model.predict(Fetuers)
                cv2.putText(Image_or_Frame,lable[0],(int(Image_or_Frame.shape[1]/2),60),
                                               cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,0,255),3
                            )
                

# test image.......

In [18]:
image_path="images\\1.jpg"

hands=mp.solutions.hands.Hands(static_image_mode=True,max_num_hands=1,min_detection_confidence=0.7)
mp_draw=mp.solutions.drawing_utils
Image=cv2.imread(image_path)
Image=cv2.resize(Image,(780, 540))
Detection_Image(Image_or_Frame=Image)

cv2.imshow('Image',Image)
k=cv2.waitKey(0) 
cv2.destroyWindow('Image')

hands.close()
del mp_draw


# function that run video or camer.....

In [19]:
def Run_Video_Or_Camer(file_or_camer=1):
    cap=cv2.VideoCapture(file_or_camer) 
    while cap.isOpened():
        
        res,frame=cap.read()
        frame=cv2.resize(frame,(780, 540))
        
        if res==False:break
        Detection_Image(Image_or_Frame=frame)
        cv2.imshow('Camer',frame) 
        
           
        k=cv2.waitKey(1) 
        if k==ord('x'):break
            
    cap.release()
    cv2.destroyWindow('Camer')
    

# test video or camer.....

In [20]:
hands=mp.solutions.hands.Hands(static_image_mode=True,max_num_hands=1,min_detection_confidence=0.5)
mp_draw=mp.solutions.drawing_utils
video_path="videos\\video1.mp4"

Run_Video_Or_Camer(file_or_camer=video_path)
hands.close()
del mp_draw


................................End Projrct......................................